In [1]:
import torch
from transformers import BertModel

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit

[2024-02-12 11:31:13,465] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
import os

# Check if CUDA is available and choose device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
!nvidia-smi

Mon Feb 12 11:31:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:89:00.0 Off |                    0 |
| N/A   35C    P0              58W / 300W |    497MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
model_path = "/home/toibazd/Data/BERT/BERT_context_pretrained_InterPro_final"

model = BertModel.from_pretrained(model_path)
model.pooler = None
model.eval()

Some weights of BertModel were not initialized from the model checkpoint at /home/toibazd/Data/BERT/BERT_context_pretrained_InterPro_final and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(544998, 256, padding_idx=0)
    (position_embeddings): Embedding(512, 256)
    (token_type_embeddings): Embedding(2, 256)
    (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=256, out_features=256, bias=True)
            (key): Linear(in_features=256, out_features=256, bias=True)
            (value): Linear(in_features=256, out_features=256, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=256, out_features=256, bias=True)
            (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [5]:
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer.pre_tokenizer = WhitespaceSplit()
tokenizer_path = "WordLevel_tokenizer_trained_InterPro.json"
tokenizer = tokenizer.from_file(tokenizer_path)
# tokenizer.enable_truncation(512)
tokenizer.get_vocab_size()


544998

In [6]:
from collections import defaultdict
from sklearn.preprocessing import MultiLabelBinarizer
import csv
from tqdm.auto import tqdm

data_dict= defaultdict()
mlb = MultiLabelBinarizer()
row_num = 0
with open("/home/toibazd/Prot2IP_GO_filtered_MF.tsv", "r") as tsvfile:
    reader = csv.reader(tsvfile, delimiter = "\t")
    for row in tqdm(reader):
        row_num+=1
        key = row[0].split("prot_")[1].split(".")[0]
        iprs = eval(row[1])
        data_dict[key] = iprs
print(row_num)
one_hot_encoded = mlb.fit_transform(data_dict.values())
one_hot_encoded_dict = {key:value for key, value in zip(data_dict.keys(), one_hot_encoded)}


0it [00:00, ?it/s]

140359


In [7]:
print(len(one_hot_encoded_dict.keys()))

139055


In [9]:
# import os
# import random

# directory = '/ibex/user/toibazd/InterPro_annotated_genomes/'
# one_hot_encoded_sentences = {}

# sentence_length = 40
# num_files = 1
# sentences_per_key = 1

# for key in tqdm(one_hot_encoded_dict):
#     if key not in one_hot_encoded_sentences:
#         one_hot_encoded_sentences[key] = []

#     file_list = os.listdir(directory)
#     random.shuffle(file_list)
#     sentences_count = 0


#     for filename in file_list[:num_files]:
#         filepath = os.path.join(directory, filename)

#         with open(filepath, 'r') as file:
#             content = file.read()
#             words = content.strip().split()

#             # Check if the content has enough words
#             if len(words) >= sentence_length:
#                 for i in range(len(words) - sentence_length + 1):
#                     # Get the key from the word at position 20
#                     current_key = words[i + 19]

#                     # Check if the current key matches the desired key
#                     if current_key == key:
#                         # Create a sentence
#                         sentence = " ".join(words[i:i + sentence_length])

#                         # Append the sentence to the list of sentences for the key
#                         one_hot_encoded_sentences[key].append(sentence)
#                         sentences_count += 1

#                         # Break if the required number of sentences per key is reached
#                         if sentences_count >= sentences_per_key:
#                             break

                            
#             if sentences_count >= sentences_per_key:
#                 break

#         if sentences_count >= sentences_per_key:
#             break




In [8]:
import json
with open('BERT_DNN_senteces.json', "r") as f:
    one_hot_encoded_sentences = json.load(f)

In [10]:
matching_string = one_hot_encoded_sentences.values()

In [11]:
len(matching_string)

139055

In [12]:
matching_string = [sentence for sublist in matching_string for sentence in sublist]
len(matching_string)

1184096

In [14]:
print(len(list(one_hot_encoded_dict.values())[1].flatten()))

3402


In [15]:
decision = True

for sentence in matching_string:
    words = sentence.split()
    if words[19] not in one_hot_encoded_dict:
        decision = False
        print(sentence)
print(decision)

True


In [16]:
for i in matching_string:
    if len(i.split()) != 40:
        print("error")
        print(len(i.split()))

In [17]:
print(len(matching_string))

1184096


In [ ]:
# Initialize an empty list to store the labels
labels_list = []

# Iterate over all sentences in your list
for sentence in sentences_list:
    # Split the sentence into words
    words = sentence.split()

    # Ensure the sentence has at least 20 words
    if len(words) >= 20:
        # Grab the word at position 20 (index 19)
        word_at_position_20 = words[19]

        # Extract the label from one_hot_encoded_dict using the word as the key
        if word_at_position_20 in one_hot_encoded_dict:
            label = one_hot_encoded_dict[word_at_position_20]

            # Append the label to the labels_list
            labels_list.append(label)
        else:
            # Handle the case where the word is not found in one_hot_encoded_dict
            # You may choose to handle this case differently based on your requirements
            labels_list.append(None)
    else:
        # Handle the case where the sentence has less than 20 words
        # You may choose to handle this case differently based on your requirements
        labels_list.append(None)


In [18]:
one_example = matching_string[0]

In [19]:
# inputs.pad(256,direction = 'right',pad_id = 3, pad_token = '[PAD]')

In [21]:
inputs = tokenizer.encode(one_example)
model.to("cuda")
input_ids = torch.tensor([inputs.ids]).to(device)
attention_mask = torch.tensor([inputs.attention_mask]).to(device)
print(input_ids)
print(attention_mask)
# Generate the embeddings
with torch.inference_mode():
    outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states = True)

hidden_states = outputs.last_hidden_state[0][20]
#     mean_tensor = torch.mean(hidden_states[1:-1], dim=0)
# embeddings_all_contexts.append(hidden_states.cpu())
    
# len(embeddings_all_contexts)

tensor([[     1,  36645,    263, 277097,  93666,  63739,   1197,    170,   2795,
          48637,  83247,  14804,   2778, 305886, 311648,  33348,  26362,  82639,
         290455, 292293, 178951,    676, 243814,  10212, 221617,  85358,   4287,
         202923,  99402,  27999, 102888, 106238,    458,   6204, 258243, 312438,
          32247, 301843, 303651, 102521, 320336,      2]], device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')


In [41]:
from bertviz import model_view
from torch.nn.functional import softmax

hidden_states_all = []
count = 0

for one_example in tqdm(matching_string):
    if count == 1:
        break
    inputs = tokenizer.encode(one_example)

    input_ids = torch.tensor([inputs.ids]).to(device)
    attention_mask = torch.tensor([inputs.attention_mask]).to(device)

    with torch.inference_mode():
        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True, output_attentions=True)

    # Extract the attentions and hidden states
    extracted_attentions = outputs.attentions[-1]
    hidden_states = outputs.last_hidden_state[0][20]
    
    # Iterate through each layer's attention matrix

    extracted_tensors = extracted_attentions[..., 1:-1, 1:-1]

        # Apply softmax to the remaining attention weights for normalization
    softmax_attention = torch.softmax(extracted_tensors, dim=-1)
    print(softmax_attention.shape)
    print(softmax_attention)
        # Now you can visualize the softmax attention using model_view
    model_view(softmax_attention, tokens)
    
    # Append hidden states and labels to the list
    hidden_states_all.append((hidden_states, one_example.split()[19]))

    count += 1


  0%|          | 0/1184096 [00:00<?, ?it/s]

torch.Size([1, 8, 40, 40])
tensor([[[[0.0281, 0.0276, 0.0267,  ..., 0.0248, 0.0248, 0.0248],
          [0.0275, 0.0265, 0.0270,  ..., 0.0248, 0.0248, 0.0248],
          [0.0254, 0.0253, 0.0258,  ..., 0.0249, 0.0249, 0.0249],
          ...,
          [0.0249, 0.0249, 0.0249,  ..., 0.0261, 0.0254, 0.0256],
          [0.0249, 0.0249, 0.0249,  ..., 0.0264, 0.0261, 0.0265],
          [0.0249, 0.0249, 0.0249,  ..., 0.0263, 0.0259, 0.0265]],

         [[0.0251, 0.0251, 0.0252,  ..., 0.0250, 0.0250, 0.0250],
          [0.0251, 0.0254, 0.0253,  ..., 0.0250, 0.0250, 0.0250],
          [0.0250, 0.0250, 0.0251,  ..., 0.0250, 0.0250, 0.0250],
          ...,
          [0.0250, 0.0250, 0.0250,  ..., 0.0256, 0.0251, 0.0251],
          [0.0250, 0.0250, 0.0250,  ..., 0.0252, 0.0252, 0.0251],
          [0.0250, 0.0250, 0.0250,  ..., 0.0251, 0.0251, 0.0251]],

         [[0.0250, 0.0251, 0.0255,  ..., 0.0250, 0.0250, 0.0250],
          [0.0250, 0.0251, 0.0252,  ..., 0.0250, 0.0250, 0.0250],
          [0.02

ValueError: The attention tensor does not have the correct number of dimensions. Make sure you set output_attentions=True when initializing your model.

In [ ]:
head_view(extracted_tensors, tokens)